In [1]:
import tensorflow_datasets as tfds

ds = tfds.load('Gigaword', split='train', shuffle_files=True, data_dir='../data')

for example in ds.take(1): # Take one example to inspect
    print(example)

2025-07-27 10:39:54.278065: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753623594.941549    8062 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753623595.107826    8062 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753623596.666278    8062 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753623596.666305    8062 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753623596.666308    8062 computation_placer.cc:177] computation placer alr

{'document': <tf.Tensor: shape=(), dtype=string, numpy=b'pakistan and kuwait on tuesday warned devastating consequences in case of war on iraq and called for averting any conflict in the region .'>, 'summary': <tf.Tensor: shape=(), dtype=string, numpy=b'kuwait pakistan warn against war on iraq'>}


2025-07-27 10:40:23.451531: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [2]:
import spacy

nlp = spacy.load('en_core_web_sm')

In [6]:
for example in ds.take(1):
    doc = nlp(example['document'].numpy().decode('utf-8'))

    for token in doc:
        print(token.lemma_.lower())

a
recent
environmental
inspection
along
the
beijing
-
guangzhou
rail
line
have
show
that
the
`
`
white
garbage
''
which
have
seriously
pollute
the
surround
region
have
almost
be
eliminate
.


2025-07-26 23:09:47.742221: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [7]:
import tensorflow as tf

def py_preprocess(document, summary):
    doc = nlp(document.numpy().decode('utf-8'))
    sum_doc = nlp(summary.numpy().decode('utf-8'))
    doc_lemmas = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    sum_lemmas = [token.lemma_ for token in sum_doc if not token.is_stop and not token.is_punct]
    return ' '.join(doc_lemmas), ' '.join(sum_lemmas)


def preprocess(row):
    doc, sum_doc = tf.py_function(
        func=py_preprocess,
        inp=[row['document'], row['summary']],
        Tout=[tf.string, tf.string]
    )
    return {'document': doc, 'summary': sum_doc}

In [8]:
preprocessed_dataset = ds.map(preprocess)

for example in preprocessed_dataset.take(1): # Take one example to inspect
    print(example)

{'document': <tf.Tensor: shape=(), dtype=string, numpy=b'foreign minister karel de gucht say sunday dalai lama decide delay trip belgium originally schedule june concern visit damage belgian tie china'>, 'summary': <tf.Tensor: shape=(), dtype=string, numpy=b'dalai lama cancel trip belgium'>}


2025-07-27 10:42:20.764304: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences